In [1]:
from pyspark.sql import SparkSession
#import pandas as pd
import numpy as np
from pyspark.sql.functions import *
#import boto3
import io

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1596734320253_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
plyr=spark.read.format("csv").load("s3://kalange-european-soccer/ply.csv",header='true')
coun = spark.read.format("csv").load("s3://kalange-european-soccer/country.csv",header='true')
league = spark.read.format("csv").load("s3://kalange-european-soccer/league.csv",header='true')
#match = spark.read.format("csv").load("s3://kalange-european-soccer/match1.csv",header='true')
team = spark.read.format("csv").load("s3://kalange-european-soccer/team.csv",header='true')
team_attr = spark.read.format("csv").load("s3://kalange-european-soccer/team_attr.csv",header='true')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
plyr_attr= spark.read.format("csv").load("s3://kalange-european-soccer/ply_attr.csv",header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
xply=plyr_attr.select(plyr_attr.columns[:7])
yply=plyr_attr.select(['id','stamina','strength'])

xply=xply.alias('xply')
yply=yply.alias('yply')

df_plyr=xply.join(yply,xply.id==yply.id).select("xply.id","player_fifa_api_id","player_api_id","date","overall_rating", \
                                               "potential","stamina","strength")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_plyr.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+-------------+----+--------------+---------+-------+--------+
| id|player_fifa_api_id|player_api_id|date|overall_rating|potential|stamina|strength|
+---+------------------+-------------+----+--------------+---------+-------+--------+
|  0|                 0|            0|   0|             0|        0|      0|       0|
+---+------------------+-------------+----+--------------+---------+-------+--------+

In [5]:
plyr.select([count(when(isnan(c), c)).alias(c) for c in plyr.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+-------------+-----------+------------------+--------+------+------+
|_c0| id|player_api_id|player_name|player_fifa_api_id|birthday|height|weight|
+---+---+-------------+-----------+------------------+--------+------+------+
|  0|  0|            0|          0|                 0|       0|     0|     0|
+---+---+-------------+-----------+------------------+--------+------+------+

In [14]:
coun.select([count(when(isnan(c), c)).alias(c) for c in coun.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+----+
|_c0| id|name|
+---+---+----+
|  0|  0|   0|
+---+---+----+

In [5]:
match= spark.read.format("csv").load("s3://kalange-european-soccer/match.csv",header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
mat=match.select(['id','country_id','league_id','season','stage','date','match_api_id','home_team_api_id','away_team_api_id',\
                'home_team_goal','away_team_goal'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
mat.select([count(when(isnan(c), c)).alias(c) for c in mat.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----------+---------+------+-----+----+------------+----------------+----------------+--------------+--------------+
| id|country_id|league_id|season|stage|date|match_api_id|home_team_api_id|away_team_api_id|home_team_goal|away_team_goal|
+---+----------+---------+------+-----+----+------------+----------------+----------------+--------------+--------------+
|  0|         0|        0|     0|    0|   0|           0|               0|               0|             0|             0|
+---+----------+---------+------+-----+----+------------+----------------+----------------+--------------+--------------+

In [7]:
"""
Dropping NaN values for the player attributes
"""


playr=df_plyr.na.drop(subset=["overall_rating"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
time = mat.select(  
                    date_format(col('date'),"yyyy-MM-dd").alias('date'),
                    dayofmonth('date').alias('day'),
                    weekofyear('date').alias('week'),
                    month('date').alias('month'),
                    year('date').alias('year'),
                    date_format('date','E').alias('weekday')).dropDuplicates()  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
matches=mat.withColumn("match_date",date_format(col('date'),"yyyy-MM-dd").cast("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
time=time.withColumn("date",col('date').cast("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
coun=coun.withColumnRenamed("id","country_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
team=team.withColumnRenamed("id","team_id")
team=team.withColumnRenamed("team_long_name","team_name")
team=team.select("team_id","team_name","team_short_name")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
time=time.withColumn('time_id',monotonically_increasing_id())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
matches=matches.withColumn('Total_Goals',col("home_team_goal")+col("away_team_goal"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
matches=matches.withColumnRenamed("home_team_api_id","home_team")
matches=matches.withColumnRenamed("away_team_api_id","away_team")
matches=matches.withColumnRenamed("home_team_goal","home_goal")
matches=matches.withColumnRenamed("away_team_goal","away_goal")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
matches.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----------+---------+---------+-----+-------------------+------------+---------+---------+---------+---------+----------+-----------+
| id|country_id|league_id|   season|stage|               date|match_api_id|home_team|away_team|home_goal|away_goal|match_date|Total_Goals|
+---+----------+---------+---------+-----+-------------------+------------+---------+---------+---------+---------+----------+-----------+
|  1|         1|        1|2008/2009|    1|2008-08-17 00:00:00|      492473|     9987|     9993|        1|        1|2008-08-17|        2.0|
|  2|         1|        1|2008/2009|    1|2008-08-16 00:00:00|      492474|    10000|     9994|        0|        0|2008-08-16|        0.0|
+---+----------+---------+---------+-----+-------------------+------------+---------+---------+---------+---------+----------+-----------+
only showing top 2 rows

In [19]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
def winning(home_t,away_t,home_g,away_g):
    if home_g>away_g:
        return home_t
    elif away_g>home_g:
        return away_t
    else :
        return "draw" 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
udf_win = F.udf(winning,StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
matches=matches.withColumn("Win_team",udf_win("home_team","away_team","home_goal","away_goal"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
matches.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----------+---------+---------+-----+-------------------+------------+---------+---------+---------+---------+----------+-----------+--------+
| id|country_id|league_id|   season|stage|               date|match_api_id|home_team|away_team|home_goal|away_goal|match_date|Total_Goals|Win_team|
+---+----------+---------+---------+-----+-------------------+------------+---------+---------+---------+---------+----------+-----------+--------+
|  1|         1|        1|2008/2009|    1|2008-08-17 00:00:00|      492473|     9987|     9993|        1|        1|2008-08-17|        2.0|    draw|
|  2|         1|        1|2008/2009|    1|2008-08-16 00:00:00|      492474|    10000|     9994|        0|        0|2008-08-16|        0.0|    draw|
|  3|         1|        1|2008/2009|    1|2008-08-16 00:00:00|      492475|     9984|     8635|        0|        3|2008-08-16|        3.0|    8635|
|  4|         1|        1|2008/2009|    1|2008-08-17 00:00:00|      492476|     9991|     9998|        5|       

In [31]:
time.createOrReplaceTempView("Time")
coun.createOrReplaceTempView("Country")
team.createOrReplaceTempView("Team")
matches.createOrReplaceTempView("Match")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
Fact_Goals = spark.sql("""
Select ti.time_id as timeid,
       c.country_id as countryid,
       c.name as country_name,
       avg(m.Total_Goals) as avg_goals
from Match m 
join Time ti
on m.match_date=ti.date
join Country c
on m.country_id=c.country_id
group by country_name,timeid,countryid
order by time_id  
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 767, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'



In [22]:
time.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1695

In [24]:
df_plyr.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- overall_rating: string (nullable = true)
 |-- potential: string (nullable = true)
 |-- stamina: string (nullable = true)
 |-- strength: string (nullable = true)